<a href="https://colab.research.google.com/github/curiosity806/2020_dacon_satellite_precipitation/blob/bogyung/Dacon_satellite_delete_ch7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import random
import sys
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dropout, MaxPooling2D, BatchNormalization, concatenate, Input
from tensorflow.keras import Model

In [0]:
# 재생산성을 위해 시드 고정
np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)

## 데이터 받아오기

In [0]:
#import os.path

#if not os.path.isfile('train.npy'):
#  !cp '/content/drive/My Drive/2020 Kaggle Study/deagwon/train_real.npy' train.npy
#if not os.path.isfile('test.npy'): 
#  !cp '/content/drive/My Drive/2020 Kaggle Study/data/test.npy' test.npy
#if not os.path.isfile('gmi_preci.npy'):
#  !cp '/content/drive/My Drive/2020 Kaggle Study/deagwon/near2_real.npy' gmi_preci.npy

train = np.load('train.npy')  # float32
test = np.load('test.npy')  # float64
gmi_preci = np.load('gmi_preci.npy')  # float32
train[:, :, :, -1] = gmi_preci.reshape(-1, 40, 40)

## Train test split

In [0]:
train, val = train_test_split(train, test_size=0.025, random_state=7777)

## Data preprocess

In [0]:
scaler = StandardScaler()
scaler.fit(train[:,:,:,:9].reshape(-1, 9))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
def preprocess(data):
    # Normalize sensor data
    data[:,:,:,:9] = scaler.transform(data[:,:,:,:9].reshape(-1, 9)).reshape(-1, 40, 40, 9)

    # Land type
    land_type_data = data[:,:,:,9]
    data[:,:,:,9] = np.where(land_type_data//100 == 2, 0.7,
                             np.where(land_type_data//100 == 3, 0.3,
                                      land_type_data//100))

    return data

In [0]:
# -9999를 포함한 이미지, 강수인 지역이 50픽셀 미만인 이미지 제거
is_valid = (train[:,:,:,-1].reshape(-1, 1600) < 0).sum(axis=1) == 0
is_valid = is_valid & ((train[:,:,:,-1].reshape(-1, 1600) >= 0.1).sum(axis=1) >= 50)
train = train[is_valid]  # (-1, 40, 40, 15)

In [0]:
train = preprocess(train)
val = preprocess(val)

## Scaling2: 0처리 후 스케일링

In [0]:
# -9999를 포함한 이미지, 강수인 지역이 50픽셀 미만인 이미지 제거
is_valid = (train[:,:,:,-1].reshape(-1, 1600) < 0).sum(axis=1) == 0
is_valid = is_valid & ((train[:,:,:,-1].reshape(-1, 1600) >= 0.1).sum(axis=1) >= 50)
train = train[is_valid]  # (-1, 40, 40, 15)

scaler2 = StandardScaler()
scaler2.fit(train[:,:,:,:9].reshape(-1, 9))

def preprocess2(data):
    # Normalize sensor data
    data[:,:,:,:9] = scaler2.transform(data[:,:,:,:9].reshape(-1, 9)).reshape(-1, 40, 40, 9)

    # Land type
    land_type_data = data[:,:,:,9]
    data[:,:,:,9] = np.where(land_type_data//100 == 2, 0.7,
                             np.where(land_type_data//100 == 3, 0.3,
                                      land_type_data//100))

    return data

train = preprocess2(train)
val = preprocess2(val)

In [11]:
for i in range(9):
  print(train[:,:,:,i].min(), train[:,:,:,i].max(), '\n')

-1.0732052 35.60011 

-1.3054495 22.093597 

-1.5137999 10.423424 

-1.4604708 6.3706517 

-2.9668453 2.7038047 

-4.8287125 8.6004305 

-1.7168117 6.2571135 

-13.262861 2.57953 

-7.2467546 1.918186 



## Rotation-flip augmentation

In [0]:
def r(a, n=1):
    return np.rot90(a, n, (1, 2))

def f_ud(a):
    return np.flip(a, 1)

def f_lr(a):
    return np.flip(a, 2)

def t(a):
    return np.transpose(a, (0, 2, 1, 3))

In [0]:
abcd = train
acbd = r(f_lr(train))
badc = f_lr(train)
bdac = r(train)
cadb = r(train, 3)
cdab = f_ud(train)
dbca = t(r(train, 2))
dcba = r(train, 2)

## 모델만들기

In [0]:
from sklearn.metrics import f1_score

def mae(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

In [0]:
def create_model():
    inputs=Input((40, 40, 9))
    
    bn=BatchNormalization()(inputs)
    conv0=Conv2D(256, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    bn=BatchNormalization()(conv0)
    conv=Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([conv0, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    conv=Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([concat, conv], axis=3)

    for i in range(5):
        bn=BatchNormalization()(concat)
        conv=Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
        concat=concatenate([concat, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    outputs=Conv2D(1, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    model=Model(inputs=inputs, outputs=outputs)
    
    return model

## 모델 학습

In [0]:
# Train without ch7

x_val = np.concatenate([val[:, :, :, :7], val[:, :, :, 8:10]], axis=3)
y_val = val[:, :, :, -1]
# x_val, y_val = val[:, :, :, :10], val[:, :, :, -1]

train_data = [[abcd, acbd, badc], [bdac, cadb, cdab], [dbca, dcba, abcd]]

In [0]:
models = []
for t in train_data:
    _train = np.concatenate(t)
    # x_train, y_train = _train[:, :, :, :10], _train[:, :, :, -1]
    x_train = np.concatenate([_train[:, :, :, :7], _train[:, :, :, 8:10]], axis=3)
    y_train = _train[:, :, :, -1]

    model = create_model()
    model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_maeOverFscore_keras', patience=4)
    #model.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_val, y_val), callbacks=[early_stopping])  # batch_size 128 넘으면 안 됨
    model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_val, y_val), callbacks=[early_stopping])  # batch_size 128 넘으면 안 됨
    models.append(model)

    del x_train, y_train
    del _train
    gc.collect()

Epoch 1/20
755/755 [==============================] - 864s 1s/step - loss: 0.2313 - maeOverFscore_keras: 1.9661 - fscore_keras: 0.6993 - val_loss: 65.8394 - val_maeOverFscore_keras: 1.9353 - val_fscore_keras: 0.6470
Epoch 2/20
755/755 [==============================] - 854s 1s/step - loss: 0.2065 - maeOverFscore_keras: 1.7144 - fscore_keras: 0.7322 - val_loss: 65.8633 - val_maeOverFscore_keras: 2.1428 - val_fscore_keras: 0.5380
Epoch 3/20
755/755 [==============================] - 849s 1s/step - loss: 0.2010 - maeOverFscore_keras: 1.6359 - fscore_keras: 0.7463 - val_loss: 65.8372 - val_maeOverFscore_keras: 1.6555 - val_fscore_keras: 0.6960
Epoch 4/20
755/755 [==============================] - 851s 1s/step - loss: 0.1988 - maeOverFscore_keras: 1.6055 - fscore_keras: 0.7522 - val_loss: 65.8298 - val_maeOverFscore_keras: 1.5107 - val_fscore_keras: 0.7268
Epoch 5/20
755/755 [==============================] - 849s 1s/step - loss: 0.1975 - maeOverFscore_keras: 1.5860 - fscore_keras: 0.7563 -

In [0]:
## 모델 저장하기
cnt = 0
for model in models:
    cnt += 1
    model.save(f'/content/drive/My Drive/2020 Kaggle Study/model/model{cnt}.h5')

## Precipatation GMI -> DPR

In [0]:
dr = [(-1, -1), (-1, 0), (-1, 1),
      (0, -1), (0, 0), (0, 1),
      (1, -1), (1, 0), (1, 1)]

def get_dist(p1, p2):  # p1, p2: shape=(-1, 2).
    x1, y1 = np.deg2rad(p1[:,0]), np.deg2rad(p1[:,1])
    x2, y2 = np.deg2rad(p2[:,0]), np.deg2rad(p2[:,1])
    dlon = x2 - x1
    dlat = y2 - y1
    a = np.sin(dlat/2)**2 + np.cos(y1) * np.cos(y2) * np.sin(dlon/2)**2 
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))    
    return 6373.0 * c  # km, shape=(-1).

# value: (40, 40, -1)
# ori_ll, tgt_ll: (40, 40, 2)
def compen_ll(value, ori_ll, tgt_ll):  # ori_ll에서의 value를 tgt_ll에 대한 값으로 바꿈
    ret = np.empty_like(value)
    n, m = value.shape[0], value.shape[1]
    for i in range(n):
        for j in range(m):
            nears = []  # (row, col, value)
            for k in range(9):
                ii = i + dr[k][0]
                jj = j + dr[k][1]
                if ii >= 0 and ii < n and jj >= 0 and jj < m:
                    nears.append((ori_ll[ii, jj][0], ori_ll[ii, jj][1],
                                  tgt_ll[i, j][0], tgt_ll[i, j][1],
                                  value[ii, jj]))
            nears = np.array(nears)  # shape=(-1, 5)
            dists = get_dist(nears[:, 0:2], nears[:, 2:4]).reshape(-1, 1)
            values = nears[:, 4].reshape(-1, 1)
            nears = np.concatenate((dists, values), 1)
            nears = nears[np.argsort(nears[:, 0])]  # sort by dist
            nears = nears[:2, :]  # 가까운 점 2개만 고려
            
            weights = 1 / (nears[:, 0] ** 2 + sys.float_info.epsilon)
            weighted_sum = (weights * nears[:, 1]).sum()
            ret[i, j] = weighted_sum / weights.sum()
    return ret

In [0]:
from multiprocessing import Process, Manager

def proc_func(splitted, dpr_preci, proc_id):
    part = splitted[proc_id]
    arr = np.empty_like(part[:, :, :, 14])  # shape=(-1, 40, 40)
    for i in range(part.shape[0]):
        arr[i, :, :] = compen_ll(part[i, :, :, 14], part[i, :, :, 10:12], part[i, :, :, 12:14])
    dpr_preci[proc_id] = arr

def gmi2dpr(test, gmi_preci):
    n_procs = 4
    procs = []
    manager = Manager()
    dpr_preci = manager.list([None] * n_procs)

    data = np.concatenate((test, gmi_preci), axis=3)
    n_imgs = data.shape[0]  # split data into n_procs arrays
    splitted = np.split(data, np.arange((n_imgs + n_procs - 1) // n_procs, n_imgs, n_imgs // n_procs))

    for proc_id in range(n_procs):
        proc = Process(target=proc_func, args=(splitted, dpr_preci, proc_id))
        proc.start()
        procs.append(proc)

    for proc in procs:
        proc.join()

    dpr_preci = np.concatenate(dpr_preci)
    return dpr_preci

In [0]:
test = preprocess(test)
x_test = test[:, :, :, :10]

preds = []
for model in models:
    pred = model.predict(x_test)
    pred = gmi2dpr(test, pred)
    preds.append(pred)
final_pred = sum(preds) / len(preds)

## submission 만들기

In [0]:
submission = pd.read_csv('/content/drive/My Drive/2020 Kaggle Study/data/sample_submission.csv')
submission.iloc[:,1:] = final_pred.reshape(-1, 1600)

In [0]:
# 제출 파일 저장하기
submission.to_csv('/content/drive/My Drive/2020 Kaggle Study/submission/submission2010.csv', index=False)
submission

,id,px_1,px_2,px_3,px_4,px_5,px_6,px_7,px_8,px_9,px_10,px_11,px_12,px_13,px_14,px_15,px_16,px_17,px_18,px_19,px_20,px_21,px_22,px_23,px_24,px_25,px_26,px_27,px_28,px_29,px_30,px_31,px_32,px_33,px_34,px_35,px_36,px_37,px_38,px_39,...,px_1561,px_1562,px_1563,px_1564,px_1565,px_1566,px_1567,px_1568,px_1569,px_1570,px_1571,px_1572,px_1573,px_1574,px_1575,px_1576,px_1577,px_1578,px_1579,px_1580,px_1581,px_1582,px_1583,px_1584,px_1585,px_1586,px_1587,px_1588,px_1589,px_1590,px_1591,px_1592,px_1593,px_1594,px_1595,px_1596,px_1597,px_1598,px_1599,px_1600
0,029858_01,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.040876,0.559549,0.113924,0.004921,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.00000,0.018626,0.017975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,029858_02,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.032283,0.18252,0.0,0.001158,0.008198,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018294,0.310187,0.121483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043242,0.381303,0.073626,0.006513,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,029858_03,0.000000,0.000000,0.00000,0.002210,0.014257,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.029154,0.013354,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000335,0.067703,0.541947,0.150564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,029858_05,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.051618,1.286749,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.883443,2.832649,4.285894,1.301341,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,029858_07,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,1.093019,1.21567,1.367476,2.318936,3.320442,3.258991,2.122521,0.854889,0.712404,1.316542,1.995333,3.580348,4.364138,4.636046,6.698931,10.597418,15.625159,19.374952,18.151189,16.942771,12.029173,6.902364,10.991206,15.371715,6.549709,1.764487,0.540584,0.138719,0.271616,0.781069,1.531339,1.901195,1.615141,1.129294,1.195447,2.205301,3.686801,3.415471,2.424429,1.676327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...